In [7]:
import neo4j
import pandas as pd
import sys
sys.path.append("../")
from utils_embeddings import load_embedding_model_std

In [8]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [9]:
model = load_embedding_model_std()

Loading embedding model: paraphrase-multilingual-MiniLM-L12-v2


In [10]:
# Generate embedding for our search query
search_query = "Richtung Hamburger Innenstadt"
query_embedding = model.encode(search_query).tolist()

In [11]:
def search_similar_entities(driver, query_embedding, search_field="name", limit=10):
    """
    Search for POIs and AOIs similar to the query embedding.
    
    Args:
        driver: Neo4j driver instance
        query_embedding: Vector embedding of the search query
        search_field: Which field to search in ("name", "description", or "tags")
        limit: Maximum number of results to return
    
    Returns:
        DataFrame containing the search results
    """
    with driver.session() as session:
        if search_field == "tags":
            # For tags search - process multiple tag embeddings
            search_query = """
            MATCH (n:POI|AOI)
            WHERE n.embeddings_json IS NOT NULL
            
            // Parse the JSON structure
            WITH n, apoc.convert.fromJsonMap(n.embeddings_json) AS embeddings
            WHERE embeddings.tags IS NOT NULL
            
            // Unwind to process each tag embedding individually
            UNWIND range(0, size(embeddings.tags) - 1) AS i
            WITH n, i, embeddings.tags[i] AS tag_vector, n.tags[i] AS tag
            
            // Calculate similarity score with each tag vector
            WITH n, i, tag, tag_vector, gds.similarity.cosine(tag_vector, $query_vector) AS tag_score
            
            // Only keep the best match per node
            WITH n, collect({tag: tag, score: tag_score, index: i}) AS tag_matches
            WITH n, apoc.coll.sortDesc(tag_matches) AS sorted_matches
            WITH n, sorted_matches[0].tag AS best_tag, sorted_matches[0].score AS best_score
            
            ORDER BY best_score DESC
            LIMIT $limit
            RETURN 
                n.name AS name, 
                n.description AS description, 
                n.tags AS tags,
                best_tag AS matched_tag,
                best_score AS score,
                labels(n) AS type,
                CASE 
                    WHEN n.geometry STARTS WITH 'POLYGON' THEN 'Polygon'
                    WHEN n.geometry STARTS WITH 'POINT' THEN 'Point'
                    ELSE 'Unknown'
                END AS geometry_type
            """
        else:
            # For name or description search, extract from embeddings_json
            search_query = """
            MATCH (n:POI|AOI)
            WHERE n.embeddings_json IS NOT NULL
            
            // Parse the JSON structure
            WITH n, apoc.convert.fromJsonMap(n.embeddings_json) AS embeddings
            WHERE embeddings[$search_field] IS NOT NULL
            
            // Calculate similarity score with the embedding vector
            WITH n, embeddings[$search_field] AS vector, gds.similarity.cosine(embeddings[$search_field], $query_vector) AS score
            
            ORDER BY score DESC
            LIMIT $limit
            RETURN 
                n.name AS name, 
                n.description AS description, 
                n.tags AS tags,
                score,
                labels(n) AS type,
                CASE 
                    WHEN n.geometry STARTS WITH 'POLYGON' THEN 'Polygon'
                    WHEN n.geometry STARTS WITH 'POINT' THEN 'Point'
                    ELSE 'Unknown'
                END AS geometry_type
            """
        
        # Execute the search with appropriate parameters
        params = {
            "query_vector": query_embedding,
            "limit": limit
        }
        
        if search_field != "tags":
            params["search_field"] = search_field
            
        results = session.run(search_query, params)
        records = [record.data() for record in results]
        
        return pd.DataFrame(records) if records else pd.DataFrame()

In [12]:
search_similar_entities(driver, query_embedding, search_field="name", limit=10)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Unknown function 'gds.similarity.cosine' (line 10, column 58 (offset: 404))
"            WITH n, embeddings[$search_field] AS vector, gds.similarity.cosine(embeddings[$search_field], $query_vector) AS score"
                                                          ^}